# Fundamentals of Machine Learning (CSCI-UA.473)

## Lab 4: Pytorch and Neural Networks 

### Perceptrons using Sklearn

In [ ]:
from sklearn.linear_model import Perceptron
from sklearn import model_selection
import matplotlib.pyplot as plt
from palmerpenguins import load_penguins
from sklearn import model_selection
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# This function returns a pandas dataframe by default (use return_X_y to get it in two numpy arrays)
penguins = load_penguins().dropna()
X = penguins[['bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g']]
y = penguins['species']
print(X.shape, y.shape)
X.head(), y.unique()

### Split the data into train and test

We'll use a 80/20 split for our training/test sets. We will not touch the test set. 

In [ ]:
# Split the data.  DO NOT TOUCH THE TEST DATA FROM HERE ON!!
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y, test_size = 0.2) # 0.2 is 20% test data.


In [ ]:
clf = Perceptron(tol=1e-3, random_state=0, shuffle=True)
### This is equivalent to SGDClassifier(loss="perceptron", eta0=1, learning_rate="constant", penalty=None)
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train_norm = (X_train - mean) / std
clf.fit(X_train_norm,y_train)

In [ ]:
clf.score((X_test- mean) / std, y_test)


## What is PyTorch?

It’s a Python based scientific computing package targeted at two sets of audiences:

-  Tensorial library that uses the power of GPUs
-  A deep learning research platform that provides maximum flexibility and speed

In this class though we will not use any GPU based computation since most of the work (labs and assignments) will be done on your laptops. 

## Import the library

In [ ]:
import torch  # <Ctrl> / <Shift> + <Return>

## The Torch Package

In [ ]:
# Generate a tensor of size 2x3x4
t = torch.Tensor(2, 3, 4)
type(t)

In [ ]:
# Get the size of the tensor
print(t.size(), t.shape)

### Vectors (1D Tensors)

In [ ]:
# Creates a 1D tensor of integers 1 to 4
v = torch.Tensor([1, 2, 3, 4])

# Create a 2x4 tensor
m = torch.Tensor([[2, 5, 3, 7],
                  [4, 2, 1, 9]])

In [ ]:
# Print number of dimensions (1D) and size of tensor
print(f'dim: {v.dim()}, size: {v.size()}')

# Print number of dimensions (2D) and size of tensor
print(f'dim: {m.dim()}, size: {m.size()}')

In [ ]:
# Element-wise multiplication
v * m

In [ ]:
# Scalar product: 1*1 + 2*0 + 3*2 + 4*0
m @ v

In [ ]:
# In-place replacement of random number from 0 to 10
x = torch.Tensor(5).random_(10)
x

In [ ]:
print(f'first: {x[0]}, last: {x[-1]}')

### Constructors

In [ ]:
# Create tensor from 3 to 8, with each having a space of 1
torch.arange(3., 8 + 1)

In [ ]:
# Create tensor from 5.7 to -2.1 with each having a space of -3
torch.arange(5.7, -2.1, -3)

In [ ]:
# returns a 1D tensor of steps equally spaced points between start=3, end=8 and steps=20
torch.linspace(3, 8, 20).view(1, -1)

In [ ]:
# Create a tensor filled with 0's
torch.zeros(3, 5)

In [ ]:
# Create a tensor filled with 1's
torch.ones(3, 2, 5)

In [ ]:
# Create a tensor with the diagonal filled with 1
torch.eye(3)

### Defining a model in pytorch

In [ ]:
import random
import torch
from torch import nn, optim
import math
from IPython import display

In [ ]:
# Define model architecture hyperparameters
D = 4  # input dimensions
C = 3  # num_classes
H = 10  # num_hidden_units

# Define training hyperparameters
learning_rate = 1e-2
lambda_l2 = 1e-1 # coefficient for the L2 regularizer. You should play with its value to see the effect of regularization

# nn package to create our linear model. Notice the Sequential container class. 
# Each Linear module has a weight and bias
# The order in which the Linear modules are defined is important as it creates the directed acyclic graph
model = nn.Sequential(
    nn.Linear(D, H),
    nn.Linear(H, C)
)

In [ ]:
# Print the model
print(model)

In [ ]:
# We need to preprocess our data and form tensors before we can use a PyTorch Model
from sklearn import preprocessing
import torch

# First move features present in pandas dataframes to tensors, this is straightforward
X_train_pth = torch.as_tensor(X_train.values, dtype=torch.float)
X_test_pth = torch.as_tensor(X_test.values, dtype=torch.float)

feature_means = torch.mean(X_train_pth, dim = 0)
feature_stds = torch.std(X_train_pth, dim=0)
print(feature_means.shape, feature_stds.shape)

# scaler = preprocessing.StandardScaler()
# X_train_pth = torch.tensor(scaler.fit_transform(X_train_pth), dtype=torch.float32)

X_train_pth = (X_train_pth - feature_means)/feature_stds

# Convert the labels need a little more care since Pytorch loss functions do not expect string labels
labels = y_train.unique() # Get all unique labels from our training set
le = preprocessing.LabelEncoder() # Define encoder using sklearn's LabelEncoder
targets = le.fit_transform(y_train) # Transform string targets to integers
# targets: array([0, 1, 2, 3, 4])

y_train_pth = torch.as_tensor(targets).long() # Finally put everything into a tensor

# Repeat for test labels
targets = le.fit_transform(y_test) # Transform string targets to integers

y_test_pth = torch.as_tensor(targets).long()

print(X_train_pth.shape,y_train_pth.shape, X_test_pth.shape, y_test_pth.shape)
print(torch.mean(X_train_pth,dim=0), torch.std(X_train_pth,dim=0))

In [ ]:
# nn package has a variety of loss functions already implemented
# we use cross entropy loss for our classification task
criterion = torch.nn.CrossEntropyLoss()

# nn package also has a variety of optimization algorithms implemented
# we use the stochastic gradient descent for our parameter updates
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=lambda_l2) # built-in L2

# Training loop
for t in range(1000):
    
    # Forward pass over the model to get the logits 
    y_pred = model(X_train_pth) #1
    # print(y_pred.shape, y_train_pth.shape)

    # Compute the loss and accuracy
    loss = criterion(y_pred, y_train_pth) #2
    
    # print(y_pred, y_train_pth)
    # break
    score, predicted = torch.max(y_pred, 1)
    acc = (y_train_pth == predicted).sum() / len(y_train)
    print("[EPOCH]: %i, [LOSS]: %.6f, [ACCURACY]: %.3f" % (t, loss.item(), acc))
    display.clear_output(wait=True)
    
    # reset (zero) the gradients before running the backward pass over the model
    # we need to do this because the gradients get accumulated at the same place across iterations
    optimizer.zero_grad() #3
    
    # Backward pass to compute the gradient of loss w.r.t our learnable params (weights and biases)
    loss.backward() #4
    
    # Update params
    optimizer.step() #5

In [ ]:
# Make predictions on the test set
X_test_pth = (X_test_pth - feature_means)/feature_stds
out = model(X_test_pth)
_, predicted = torch.max(out.data, 1)

#get accuration
print('Accuracy of the network %.4f %%' % (100 * torch.sum(y_test_pth==predicted).double() / len(y_test_pth)))


### Classification of non-linearly separable dataset using multi-layer perceptron
We will not demonstrate how to build a linear model and a neural network model using PyTorch and how to train it to classify a toy dataset which is not linearly separable. 

Import appropriate packages

In [ ]:
# some predefined helper functions provided for plotting data and model outputs
from plot_lib import plot_data, plot_model, set_default

In [ ]:
set_default()

In [ ]:
seed = 12345
random.seed(seed)
torch.manual_seed(seed)
N = 1000  # num_samples_per_class
D = 2  # dimensions
C = 3  # num_classes
H = 100  # num_hidden_units

### Create the dataset
We will now create a data set consisting of three classes and is in the shape of a spiral

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
X = torch.zeros(N * C, D).to(device)
y = torch.zeros(N * C, dtype=torch.long).to(device)
for c in range(C):
    index = 0
    t = torch.linspace(0, 1, N)
    # When c = 0 and t = 0: start of linspace
    # When c = 0 and t = 1: end of linpace
    # This inner_var is for the formula inside sin() and cos() like sin(inner_var) and cos(inner_Var)
    inner_var = torch.linspace(
        # When t = 0
        (2 * math.pi / C) * (c),
        # When t = 1
        (2 * math.pi / C) * (2 + c),
        N
    ) + torch.randn(N) * 0.2
    
    for ix in range(N * c, N * (c + 1)):
        X[ix] = t[index] * torch.FloatTensor((
            math.sin(inner_var[index]), math.cos(inner_var[index])
        ))
        y[ix] = c
        index += 1

print("Shapes:")
print("X:", tuple(X.size()))
print("y:", tuple(y.size()))

In [ ]:
# visualise the data using the plot_data function provided as a helper function
plot_data(X, y)

### Linear Model
We now define a linear classification model using PyTorch and train it using stochastic gradient descent with the help of the autograd package. 

Initialize some hyper-parameter values, such as, learning rate, regularization coefficient etc

In [ ]:
learning_rate = 1e-2
lambda_l2 = 1e-3 # coefficient for the L2 regularizer. You should play with its value to see the effect of regularization

Create the linear model and print it

In [ ]:
# nn package to create our linear model. Notice the Sequential container class. 
# Each Linear module has a weight and bias
# The order in which the Linear modules are defined is important as it creates the directed acyclic graph
model = nn.Sequential(
    nn.Linear(D, H),
    nn.Linear(H,H),
    nn.Linear(H, C)
)

In [ ]:
# Print the model
print(model)

Create the training loop and print the metrics (loss and accuracy) at the end of each iteration

In [ ]:
# nn package has a variety of loss functions already implemented
# we use cross entropy loss for our classification task
criterion = torch.nn.CrossEntropyLoss()

# nn package also has a variety of optimization algorithms implemented
# we use the stochastic gradient descent for our parameter updates
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=lambda_l2) # built-in L2

# Training loop
for t in range(1000):
    
    # Forward pass over the model to get the logits 
    y_pred = model(X) #1
    
    # Compute the loss and accuracy
    loss = criterion(y_pred, y) #2
    # score, predicted = torch.max(y_pred, 1)
    # acc = (y == predicted).sum().float() / len(y)
    print("[EPOCH]: %i, [LOSS]: %.6f, [ACCURACY]: %.3f" % (t, loss.item(), acc))
    display.clear_output(wait=True)
    
    # reset (zero) the gradients before running the backward pass over the model
    # we need to do this because the gradients get accumulated at the same place across iterations
    optimizer.zero_grad() #3
    
    # Backward pass to compute the gradient of loss w.r.t our learnable params (weights and biases)
    loss.backward() #4
    
    # Update params
    optimizer.step() #5

Plot the output of the model (in this case a collection of hyper-planes)

In [ ]:
# Plot trained model
plot_model(X, y, model)

### Two layer neural network
We now define a two layer (single hidden layer) neural network model using PyTorch and train it using stochastic gradient descent with the help of the autograd package. 

Initialize the hyper-parameters like before. Play around with the learning rate and the regularization parameter to see their effect on the optimization. 

In [ ]:
learning_rate = 1e-2
lambda_l2 = 1e-3

Create and print the two layer MLP with ReLU as the activation units

In [ ]:
# nn package to create our linear model
# each Linear module has a weight and bias

model = nn.Sequential(
    nn.Linear(D, H),
    nn.ReLU(),
    nn.Linear(H,C)
    
)

print(model)

Create the training loop and print the metrics (loss and accuracy) at the end of each iteration

In [ ]:
# nn package has a variety of loss functions already implemented
# we use cross entropy loss for our classification task
criterion = torch.nn.CrossEntropyLoss()

# nn package also has a variety of optimization algorithms implemented
# we use the stochastic gradient descent for our parameter updates
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=lambda_l2) # built-in L2

# e = 1.  # plotting purpose

# Training
for t in range(1000):
    
    # Forward pass over the model to get the logits
    y_pred = model(X)
    
    # Compute the loss and accuracy
    loss = criterion(y_pred, y)
    score, predicted = torch.max(y_pred, 1)
    acc = (y == predicted).sum().float() / len(y)
    print("[EPOCH]: %i, [LOSS]: %.6f, [ACCURACY]: %.3f" % (t, loss.item(), acc))
    display.clear_output(wait=True)
    
    # reset (zero) the gradients before running the backward pass over the model
    # we need to do this because the gradients get accumulated at the same place across iterations
    optimizer.zero_grad()
    
    # Backward pass to compute the gradient of loss w.r.t our learnable params (weights and biases)
    loss.backward()
    
    # Update params
    optimizer.step()

In [ ]:
# Plot trained model
plot_model(X, y, model)

## Regression

We will now demonstrate how to build a linear model and a neural network model for a regression task using PyTorch.  

In [ ]:
import random
import torch
from torch import nn, optim
import math
from IPython import display
from plot_lib import plot_data, plot_model, set_default
from matplotlib import pyplot as plt

In [ ]:
set_default()

### Create the data

In [ ]:
seed = 1
random.seed(seed)
torch.manual_seed(seed)
N = 1000  # num_samples_per_class
D = 1  # dimensions
C = 1  # num_classes
H = 100  # num_hidden_units

In [ ]:
X = torch.unsqueeze(torch.linspace(-1, 1, 100), dim=1).to(device)
y = X.pow(3) + 0.3 * torch.rand(X.size()).to(device)

In [ ]:
print("Shapes:")
print("X:", tuple(X.size()))
print("y:", tuple(y.size()))

In [ ]:
plt.scatter(X.cpu().numpy(), y.cpu().numpy())
plt.axis('equal');

### Linear model

Initialize the values of the hyper-parameters

In [ ]:
learning_rate = 1e-3
lambda_l2 = 1e-5

Create the model and print it

In [ ]:
# nn package to create our linear model
# each Linear module has a weight and bias
model = nn.Sequential(
    nn.Linear(D, H),
    nn.Linear(H, C)
)

# print the model
print(model)

Create the training loop

In [ ]:
# we use MSE (mean squared error) loss from the nn package for our regression task
criterion = torch.nn.MSELoss()

# we use the optim package to apply stochastic gradient descent for our parameter updates
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=lambda_l2) # built-in L2

# Training loop
for t in range(1000):
    
    # forward pass over the model to get the logits (inputs to the loss function)
    y_pred = model(X)
    
    # Compute the loss (MSE)
    loss = criterion(y_pred, y)
    print("[EPOCH]: %i, [LOSS or MSE]: %.6f" % (t, loss.item()))
    display.clear_output(wait=True)
    
    # zero the gradients before running the backward pass
    optimizer.zero_grad()
    
    # Backward pass to compute the gradient of loss w.r.t our learnable params 
    loss.backward()
    
    # Update params
    optimizer.step()

In [ ]:
# Plot trained model
plt.scatter(X.data.cpu().numpy(), y.data.cpu().numpy())
plt.plot(X.data.cpu().numpy(), y_pred.data.cpu().numpy(), 'r-', lw=5)
plt.axis('equal');

Play around with the values of the hyper-parameters and the value of H (number of hidden units) to observe the change in the models being learnt

## Convolutional Neural Networks (CNNs)

We will test the following assumptions pertaining to CNNs 

* Compositionality obtained using many layers
* Locality + stationarity of images assumed by the convolutional layers
* Invariance of object class to translations assumed by the pooling layers

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy
from plot_lib import plot_data, plot_model, set_default

set_default()
# Get our device in a variable
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# function to count number of parameters
def get_n_params(model):
    np=0
    for p in list(model.parameters()):
        np += p.nelement()
    return np

### Load the Dataset (MNIST)

Load the MNIST handwritten digits dataset. We can use the PyTorch DataLoader utilities for this. This will download, shuffle, normalize data and arrange it in batches. Normalizing involves subtracting some coefficient (usually the mean) from each pixel values and dividing the resulting pixel values by another coefficient (usually the variance of the original pixel values). We also display some images.

In [ ]:
input_size  = 28*28   # images are 28x28 pixels
output_size = 10      # there are 10 classes

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=True)

In [ ]:
# show some images
plt.figure(figsize=(16, 6))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    image, _ = train_loader.dataset.__getitem__(i)
    plt.imshow(image.squeeze().numpy(), cmap='gray')
    plt.axis('off');

### Create the model classes
For comparison purposes we will create two models classes: 
1. Multi-layer Perceptron
2. Convolutional Neural Network

Pay special attention to the order of the layer while creating CNNs.

In [ ]:
class FC2Layer(nn.Module):
    def __init__(self, input_size, n_hidden, output_size, activation='relu'):
        super(FC2Layer, self).__init__()
        self.input_size = input_size
        if activation =='relu':
            self.activation = nn.ReLU()  
        elif activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        else :
            self.activation = nn.ReLU()
            print("Activation function not implemented, using default ReLU")
        self.network = nn.Sequential(
            nn.Linear(input_size, n_hidden), 
            self.activation,
            nn.Linear(n_hidden, n_hidden), 
            self.activation,
            nn.Linear(n_hidden, output_size), 
            nn.LogSoftmax(dim=1))

    def forward(self, x):
        x = x.view(-1, self.input_size)      
        return self.network(x)
    
class CNN(nn.Module):
    def __init__(self, input_size, n_feature, output_size, activation='relu'):
        super(CNN, self).__init__()
        self.n_feature = n_feature
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=n_feature, kernel_size=5)
        self.conv2 = nn.Conv2d(n_feature, n_feature, kernel_size=5)
        self.fc1 = nn.Linear(n_feature*4*4, 50)
        self.fc2 = nn.Linear(50, 10)
        if activation =='relu':
            self.activation = nn.ReLU()  
        elif activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        else :
            self.activation = nn.ReLU()
            print("Activation function not implemented, using default ReLU")
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.activation(x) # Will it make a difference if we apply the non-linearity after the pooling layer?
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv2(x)
        x = self.activation(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = x.view(-1, self.n_feature*4*4) # this is where are flattening the 2D feature maps into a single 1D vector so as to be used by the subsequent fully connected layer
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

In [ ]:
def train(epoch, model, optimizer, perm=torch.arange(0, 784).long(), verbose=False):
    model.train()
    epoch_loss = 0
    losses = []
    for batch_idx, (data, target) in enumerate(train_loader):
        # send data to device, where the "device" is either a GPU if it exists or a CPU
        data, target = data.to(device), target.to(device)
        
        # permute pixels
        data = data.view(-1, 28*28)
        data = data[:, perm]
        data = data.view(-1, 1, 28, 28)

        optimizer.zero_grad()
        # forward pass through the model
        output = model(data)
        # forward pass through the cross-entropy loss function
        loss = F.nll_loss(output, target)
        # backward pass through the cross-entropy loss function and the model
        loss.backward()
        
        optimizer.step()
        if batch_idx % 100 == 0:
            losses.append(loss.detach())
            if verbose :
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))
    return losses

def test(model, perm=torch.arange(0, 784).long(), verbose=False):
    model.eval()
    accuracy_list = []
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            # send data to device, where the "device" is either a GPU if it exists or a CPU
            data, target = data.to(device), target.to(device)

            # permute pixels
            data = data.view(-1, 28*28)
            data = data[:, perm]
            data = data.view(-1, 1, 28, 28)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss                                                               
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability                                                                 
            correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()

        test_loss /= len(test_loader.dataset)
        accuracy = 100. * correct / len(test_loader.dataset)
        accuracy_list.append(accuracy) 
        if verbose :
            print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
                test_loss, correct, len(test_loader.dataset),
                accuracy))
    return test_loss
        

### Train a small MLP

In [ ]:
n_hidden = 8 # number of hidden units
model_fnn = FC2Layer(input_size, n_hidden, output_size)
model_fnn = model_fnn.to(device)

optimizer = optim.SGD(model_fnn.parameters(), lr=0.01, momentum=0.5)
print('Number of parameters: {}'.format(get_n_params(model_fnn)))
for epoch in range(0, 2):
    train(epoch, model_fnn, optimizer, verbose=True)
    test(model_fnn, verbose=True)

### Train a ConvNet with the same number of parameters
Play around with the hyper-parameters to understand their relationship with model performance.

In [ ]:
# Training settings 
n_features = 6 # number of feature maps

model_cnn = CNN(input_size, n_features, output_size, activation='relu')
model_cnn.to(device)
optimizer = optim.SGD(model_cnn.parameters(), lr=0.01, momentum=0.5)
print('Number of parameters: {}'.format(get_n_params(model_cnn)))

for epoch in range(0, 2):
    train(epoch, model_cnn, optimizer, verbose=True)
    test(model_cnn, verbose=True)

### ConvNet performs better with the same number of parameters, thanks to its use of prior knowledge about images

* Use of convolution: Locality and stationarity in images
* Pooling: builds in some translation invariance

### What happens if the assumptions are no longer true?
Let us break the assumption of locality and permute the pixel within each image using an arbitrary permutation matrix. Also display the permuted images.

In [ ]:
perm = torch.randperm(784)
plt.figure(figsize=(16, 12))
for i in range(10):
    image, _ = train_loader.dataset.__getitem__(i)
    # permute pixels
    image_perm = image.view(-1, 28*28).clone()
    image_perm = image_perm[:, perm]
    image_perm = image_perm.view(-1, 1, 28, 28)
    plt.subplot(4, 5, i + 1)
    plt.imshow(image.squeeze().numpy(), cmap='gray')
    plt.axis('off')
    plt.subplot(4, 5, i + 11)
    plt.imshow(image_perm.squeeze().numpy(), cmap='gray')
    plt.axis('off')

### CNNs with permuted pixels
What do you think will happen to CNNs when given permuted pixels as inputs? 

In [ ]:
# Training settings 
n_features = 6 # number of feature maps


model_cnn = CNN(input_size, n_features, output_size)
model_cnn.to(device)
optimizer = optim.SGD(model_cnn.parameters(), lr=0.01, momentum=0.5)
print('Number of parameters: {}'.format(get_n_params(model_cnn)))

for epoch in range(0, 3):
    train(epoch, model_cnn, optimizer, perm, verbose=True)
    test(model_cnn, perm, verbose=True)

### MLPs with permuted pixels

In [ ]:
n_hidden = 8    # number of hidden units

model_fnn = FC2Layer(input_size, n_hidden, output_size)
model_fnn.to(device)
optimizer = optim.SGD(model_fnn.parameters(), lr=0.01, momentum=0.5)
print('Number of parameters: {}'.format(get_n_params(model_fnn)))

for epoch in range(0, 2):
    train(epoch, model_fnn, optimizer, perm, verbose=True)
    test(model_fnn, perm, verbose=True)

### Activation Functions
When we moved to Neural Networks we switched to ReLU as our activation function (or non-linearity). Recall that in logistic regression our non-linearty was the Sigmoid function. The sigmoid activation has a problem of vanishing gradients i.e. the gradients we get during backpropagation as often very close to 0. Think about what problem will this cause? In the next cell, we try both activations on our FC network and compare how the training proceeds.

In [ ]:
n_hidden = 8 # number of hidden units
model_fnn_1 = FC2Layer(input_size, n_hidden, output_size, activation='relu')
model_fnn_1 = model_fnn_1.to(device)
model_fnn_2 = FC2Layer(input_size, n_hidden, output_size, activation='sigmoid')
model_fnn_2 = model_fnn_2.to(device)

optimizer_1 = optim.SGD(model_fnn_1.parameters(), lr=0.01, momentum=0.5)
optimizer_2 = optim.SGD(model_fnn_2.parameters(), lr=0.01, momentum=0.5)
print('Number of parameters: {}'.format(get_n_params(model_fnn_1)))
train_losses_1 = []
train_losses_2 = []
for epoch in range(0, 15):
    epoch_losses = train(epoch, model_fnn_1, optimizer_1)
    train_losses_1.extend(epoch_losses)
    epoch_losses = train(epoch, model_fnn_2, optimizer_2)
    train_losses_2.extend(epoch_losses)
plt.plot(train_losses_1, label='ReLU')
plt.plot(train_losses_2, label='Sigmoid')
plt.legend()
plt.show()

Think about why does the Sigmoid function have this issue but not the ReLU?

### Using Dropout to prevent overfitting
Dropout is a regularization technique which is now widely used to train deep neural networks. The idea is to randomly dropout some activations while training the models (Why would this help?). While the effect of using dropout is demonstrable for models on larger datasets (which takes longer to train!), we use a toy regression problem to demonstrate how dropout prevents overfitting.

In [ ]:
N = 50 #number of data points
noise = 0.3

#generate the train data
X_train = torch.unsqueeze(torch.linspace(-1, 1, N),1)
Y_train = X_train + noise * torch.normal(torch.zeros(N,1), torch.ones(N,1))

#generate the test data
X_test = torch.unsqueeze(torch.linspace(-1,1,N),1)
Y_test = X_test + noise * torch.normal(torch.zeros(N,1), torch.ones(N,1))

In [ ]:
#create a neural network with out dropout
N_h = 100 #hidden nodes

model = torch.nn.Sequential(
    nn.Linear(1, N_h),
    nn.ReLU(),
    nn.Linear(N_h, N_h),
    nn.ReLU(),
    nn.Linear(N_h, 1)
)

#create a network with dropout
model_dropout = nn.Sequential(
    nn.Linear(1, N_h),
    nn.Dropout(0.5), #50 % probability 
    nn.ReLU(),
    torch.nn.Linear(N_h, N_h),
    torch.nn.Dropout(0.2), #20% probability
    torch.nn.ReLU(),
    torch.nn.Linear(N_h, 1),
)

plt.scatter(X_train.data.numpy(), Y_train.data.numpy(), c='purple', alpha=0.5, label='train')
plt.scatter(X_test.data.numpy(), Y_test.data.numpy(), c='yellow', alpha=0.5, label='test')
plt.legend()
plt.show()

In [ ]:
# Define number of hidden layers
N_h = 100

# Define a FC model
model = torch.nn.Sequential(
    torch.nn.Linear(1, N_h),
    torch.nn.ReLU(),
    torch.nn.Linear(N_h, N_h),
    torch.nn.ReLU(),
    torch.nn.Linear(N_h, 1),
)

# Define a FC model with dropout -> simply add nn.Dropout(p) where p is the probability of randomly dropping out a node 
model_dropout = torch.nn.Sequential(
    torch.nn.Linear(1, N_h),
    torch.nn.Dropout(0.2),
    torch.nn.ReLU(),
    torch.nn.Linear(N_h, N_h),
    torch.nn.Dropout(0.2),
    torch.nn.ReLU(),
    torch.nn.Linear(N_h, 1),
)

In [ ]:
opt = torch.optim.Adam(model.parameters(), lr=0.01)
opt_dropout = torch.optim.Adam(model_dropout.parameters(), lr=0.01)
loss_fn = torch.nn.MSELoss()

In [ ]:
max_epochs = 1000

# Training blueprint - we are not using minibatches since our data is small enough to use it completely
for epoch in range(max_epochs):
    pred = model(X_train) 
    loss = loss_fn(pred, Y_train)
    opt.zero_grad()
    loss.backward()
    opt.step()
    
    pred_dropout = model_dropout(X_train)
    loss_dropout = loss_fn(pred_dropout, Y_train)
    opt_dropout.zero_grad()
    loss_dropout.backward()
    opt_dropout.step()
    
    
    if epoch % 100 == 0:
        
        model.eval()
        model_dropout.eval()
        
        test_pred = model(X_test)
        test_loss = loss_fn(test_pred, Y_test)
        
        test_pred_dropout = model_dropout(X_test)
        test_loss_dropout = loss_fn(test_pred_dropout, Y_test)
        
        plt.scatter(X_train.data.numpy(), Y_train.data.numpy(), c='purple', alpha=0.5, label='train')
        plt.scatter(X_test.data.numpy(), Y_test.data.numpy(), c='yellow', alpha=0.5, label='test')
        plt.plot(X_test.data.numpy(), test_pred.data.numpy(), 'r-', lw=3, label='normal')
        plt.plot(X_test.data.numpy(), test_pred_dropout.data.numpy(), 'b--', lw=3,  label='dropout')
        
        plt.title('Epoch %d, Loss = %0.4f, Loss with dropout = %0.4f' % (epoch, test_loss, test_loss_dropout))
        
        plt.legend()

        model.train()
        model_dropout.train()
        
        plt.pause(0.05)